# Agente Auto


This notebook presents an agent-based model that simulates the propagation of a disease through a network.
It demonstrates how to use the [agentpy](https://agentpy.readthedocs.io) package to create and visualize networks, use the interactive module, and perform different types of sensitivity analysis. 

In [1]:
# Model design
import agentpy as ap
import networkx as nx 
import random 

# Visualization
import matplotlib.pyplot as plt 
import seaborn as sns
import IPython

## About the model 
Es



## Defining the model

In [170]:
class Car(ap.Agent):
    
    def setup(self):  
        """ Initialize a new variable at agent creation. """
        self.condition = 0  # Conduciendo = 0, Standby = 1
        rgb = self.model.random
        self.turn_chance = rgb.random()
        self.turnDir = rgb.random()
        self.grid = self.model.grid
        self.dir = 0 # Horizontal = 0, Vertical = 1
        self.side = 0 # Right/Down = 0, Left/Up = 1
        self.id = 0
        
        
        
    def drive(self):
        """ Manejar """
        ### Setters iniciales
        if (self.grid.positions[self] == (0, int(self.p.gridsize/2))):
            self.dir = 1
        elif (self.grid.positions[self] == (0, int(self.p.gridsize/2+1))):
            self.dir = 1
            self.side = 1
        elif (self.grid.positions[self] == ( int(self.p.gridsize/2), 0)):
            self.side = 1

        ### Lado de la calle
        if self.side == 0: #Right      
            if self.condition == 0 and self.dir == 0:  #Horizontal
                self.grid.move_by(self, (0,1))
            elif self.condition == 0 and self.dir == 1: #Vertical
                self.grid.move_by(self, (1,0))
        else: #Left
            if self.condition == 0 and self.dir == 0:  #Horizontal
                self.grid.move_by(self, (0,-1))
            elif self.condition == 0 and self.dir == 1: #Vertical
                self.grid.move_by(self, (-1,0))

    ### Girar
        if ((self.grid.positions[self] == (self.p.gridsize/2,self.p.gridsize/2)
             and self.dir == 1 and self.side == 0)and  # street is Turnable (V-R)           Vertical-Abajo
            self.turn_chance > self.turnDir):
            self.dir = 0
            self.side = 1
        elif ((self.grid.positions[self] == (self.p.gridsize/2,self.p.gridsize/2+1)
             and self.dir == 0 and self.side == 1)and  # street is Turnable (H-L)           Horizontal-Izquierda
            self.turn_chance > self.turnDir):
            self.dir = 1
        elif ((self.grid.positions[self] == (self.p.gridsize/2+1,self.p.gridsize/2)
             and self.dir == 0 and self.side == 0) and  # street is Turnable (H-R)          Horizontal-Derecha
            self.turn_chance > self.turnDir):
            self.dir = 1
        elif ((self.grid.positions[self] == (self.p.gridsize/2+1,self.p.gridsize/2+1)
             and self.dir == 1 and self.side == 1)and  # street is Turnable (V-L)           Vertical-Arriba
            self.turn_chance > self.turnDir):
            self.dir = 0
            self.side = 0
        
       
    def stop(self):
        self.grid.move_by(self, (0,1))

Traffic Light

In [192]:
class TLight(ap.Agent):
    
    def setup(self):
        self.condition = 2          # Rojo = 2, Verde = 3
        self.grid = self.model.grid
        self.id = 0

    #def pos(self):
        #self.pos = self.grid.pos[self]

    def lightRed(self):
        self.condition = 2

    def lightGreen(self):
        self.condition = 3
    

In [198]:
class CarModel(ap.Model):
    
    def setup(self):

        gridsize = self.p['gridsize']

        rng = self.model.random
        """ Initialize the agents and network of the model. """
        self.grid = ap.Grid(self, (self.p.gridsize, self.p.gridsize), torus= True, track_empty= True)

        # self.agents = Car
        self.agents = ap.AgentList(self, 1, Car)
        if 0.5 > rng.random():
            self.grid.add_agents(self.agents, [(int(self.p.gridsize/2),0)])
        else:
            self.grid.add_agents(self.agents, [(int(self.p.gridsize/2+1),0)])

        count_id = 0
        for car in self.agents:
            car.id = count_id
            count_id += 1

        # self.semaforo = TLight
        self.semaforo = ap.AgentList(self, 4, TLight)
        self.grid.add_agents(self.semaforo, positions = [
            (int((gridsize/2)-1), int((gridsize/2)+2)), # Arriba-Derecha
            (int((gridsize/2)+2), int((gridsize/2)-1)), # Abajo-Izquierda
            (int((gridsize/2)-1), int((gridsize/2)-1)), # Arriba-Izquierda
            (int((gridsize/2)+2), int((gridsize/2)+2))  # Abajo-Derecha
        ])

        count_id = 0
        for tf in self.semaforo:
            tf.id = count_id
            count_id += 1
        
        
    def step(self):   
        """ Define the models' events per simulation step. """
        
        countRight = 0
        countLeft = 0
        countUp = 0
        countDown = 0

        '''
        self.dir = 0 # Horizontal = 0, Vertical = 1
        self.side = 0 # Right/Down = 0, Left/Up = 1
        
        self.grid.add_agents(self.semaforo, positions = [
            (int((gridsize/2)-1), int((gridsize/2)+2)), # Arriba-Derecha
            (int((gridsize/2)+2), int((gridsize/2)-1)), # Abajo-Izquierda
            (int((gridsize/2)-1), int((gridsize/2)-1)), # Arriba-Izquierda
            (int((gridsize/2)+2), int((gridsize/2)+2))  # Abajo-Derecha
        ])
        '''

        for car in self.agents:
            if (car.dir == 1 and car.side == 0 and car.p.turn_chance > car.turnDir and self.grid.positions[car] == (self.p.gridsize/2,self.p.gridsize/2)) or (car.dir == 0 and car.side == 1 and car.turn_chance < car.turnDir):
                countLeft += 1
            elif (car.dir == 0 and car.side == 1 and car.p.turn_chance > car.turnDir and self.grid.positions[car] == (self.p.gridsize/2,self.p.gridsize/2)) or (car.dir == 1 and car.side == 1 and car.turn_chance < car.turnDir):
                countUp += 1
            elif (car.dir == 0 and car.side == 0 and car.p.turn_chance > car.turnDir and self.grid.positions[car] == (self.p.gridsize/2,self.p.gridsize/2)) or (car.dir == 1 and car.side == 0 and car.turn_chance < car.turnDir):
                countDown += 1
            elif (car.dir == 1 and car.side == 1 and car.p.turn_chance > car.turnDir and self.grid.positions[car] == (self.p.gridsize/2,self.p.gridsize/2)) or (car.dir == 0 and car.side == 0 and car.turn_chance < car.turnDir):
                countRight += 1

        trafficLight = self.semaforo

        if ( (countRight > countLeft or countRight == countLeft) and (countRight > countUp or countRight == countUp) and (countRight > countDown or countRight == countDown) ):
            # Se cambia el estado de color del semaforo
            trafficLight[3].lightGreen()
            trafficLight[0].lightRed()
            trafficLight[1].lightRed()
            trafficLight[2].lightRed()
            # Se detienen los carros correspondientes.
            for car in self.agents:
                if (car.dir == 1 and car.side == 1 and car.turn_chance > car.turnDir and self.grid.positions[car][1] <= self.p.gridsize/2) or (car.dir == 0 and car.side == 0 and car.turn_chance < car.turnDir):
                    car.drive()
                else:
                    car.stop()
        elif ( (countLeft > countRight or countLeft == countRight) and (countLeft > countUp or countLeft == countUp) and (countLeft > countDown or countLeft == countDown) ):
            # Se cambia el estado de color del semaforo
            trafficLight[2].lightGreen()
            trafficLight[0].lightRed()
            trafficLight[1].lightRed()
            trafficLight[3].lightRed()
            # Se detienen los carros correspondientes.
            '''for car in self.agents:
                if (car.dir == 1 and car.side == 0 and car.turn_chance > car.turnDir and self.grid.positions[car] == (self.p.gridsize/2,self.p.gridsize/2)) or (car.dir == 0 and car.side == 1 and car.turn_chance < car.turnDir):
                    car.drive()
                else:
                    car.stop()'''
        elif ( (countUp > countRight or countUp == countRight) and (countUp > countLeft or countUp == countLeft) and (countUp > countDown or countUp == countDown) ):
            # Se cambia el estado de color del semaforo
            trafficLight[0].lightGreen()
            trafficLight[1].lightRed()
            trafficLight[2].lightRed()
            trafficLight[3].lightRed()
           # Se detienen los carros correspondientes.
            '''for car in self.agents:
                if (car.dir == 1 and car.side == 0 and car.p.turn_chance > car.turnDir and self.grid.positions[car] == (self.p.gridsize/2,self.p.gridsize/2)) or (car.dir == 0 and car.side == 1 and car.turn_chance < car.turnDir):
                    car.drive()
                else:
                    car.stop()'''
        elif ( (countDown > countRight or countDown == countRight) and (countDown > countLeft or countDown == countLeft) and (countDown > countUp or countDown == countUp) ):
            # Se cambia el estado de color del semaforo
            trafficLight[1].lightGreen()
            trafficLight[0].lightRed()
            trafficLight[2].lightRed()
            trafficLight[3].lightRed()
            # Se detienen los carros correspondientes.
            '''for car in self.agents:
                if (car.dir == 0 and car.side == 1 and car.p.turn_chance > car.turnDir and self.grid.positions[car] == (self.p.gridsize/2,self.p.gridsize/2)) or (car.dir == 1 and car.side == 1 and car.turn_chance < car.turnDir):
                    car.drive()
                else:
                    car.stop()'''



        self.agents.drive()
        rng = self.model.random
        if self.p.car_chance > rng.random() and len(self.agents) < self.p.cars: 
            self.new_agents = ap.AgentList(self, 1, Car)
            self.agents.extend(self.new_agents)
            if 0.5 > rng.random():
                if 0.5 > rng.random():
                    self.grid.add_agents(self.new_agents, [(int(self.p.gridsize/2),0)], empty=True)
                else:
                    self.grid.add_agents(self.new_agents, [(int(self.p.gridsize/2+1),0)], empty=True)
            else:
                if 0.5 > rng.random():
                    self.grid.add_agents(self.new_agents, [(0, int(self.p.gridsize/2))], empty=True)
                    
                else:
                    self.grid.add_agents(self.new_agents, [(0, int(self.p.gridsize/2+1))], empty=True)   

         

    
        

## Running a simulation

In [195]:
parameters = {  
    'gridsize': 24,
    'cars': 9,
    'steps': 200,
    'turn_chance': 0.5,
    'car_chance': 0.1,
}

model = CarModel(parameters)
results = model.run()

Completed: 200 steps
Run time: 0:00:00.038936
Simulation finished


## Analyzing results

In [196]:
results

DataDict {
'info': Dictionary with 9 keys
'parameters': 
    'constants': Dictionary with 5 keys
'reporters': DataFrame with 1 variable and 1 row
}

In [199]:
def animation_plot(model, ax):
    attr_grid = model.grid.attr_grid('condition')
    # 0: carro driving
    # 1: carro standby
    # 2: semáforo rojo
    # 3: semáforo verde
    # 4: pasto
    color_dict = {0: '#000000', 1: '#000000', 2: '#D62C2C', 3: '#21D41E', 4: '#669301', None: '#EAE5E2'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Car model \n Time-step: {model.t}")

fig, ax = plt.subplots()
model = CarModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())